In [ ]:

import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data = pd.read_csv('/content/IMDB_Dataset_colab2.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
english_stops = set(stopwords.words('english'))

In [ ]:
def load_dataset():
    df = pd.read_csv('/content/IMDB_Dataset_colab2.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [ ]:
print(x_data.dtypes)
print(y_data.dtypes)

object
int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
46108    [i, remember, terrified, movie, blood, i, youn...
15641    [i, remembered, cartoon, years, staying, power...
22919    [i, consider, film, best, one, mike, hammer, b...
46764    [dolemite, awesome, rudy, ray, moore, rhymin, ...
37818    [this, first, time, i, feel, need, comment, im...
                               ...                        
31575    [i, thought, harvey, keitel, young, fresh, sex...
36253    [gespenster, forms, together, yella, jerichow,...
3301     [famous, british, film, bad, given, away, free...
9610     [i, used, write, comments, imdb, i, anymore, i...
46285    [although, one, vonnegut, better, known, works...
Name: review, Length: 40000, dtype: object 

19176    [i, truly, enjoyed, movie, however, i, realize...
12934    [on, march, man, found, dead, toilets, manhatt...
41211    [i, curious, rifle, beckett, using, movie, als...
8535     [i, lucky, enough, catch, movie, volunteering,...
24787    [the, photography, bid, budget, production, su...
 

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

In [ ]:
type(x_train)

pandas.core.series.Series

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    8     1  9580 ... 14046  6696   577]
 [  564  1160    44 ...   853  5740  4431]
 [    8    20  2669 ...     0     0     0]
 ...
 [  487   354   523 ...     0     0     0]
 [    7     3   794 ...     0     0     0]
 [   40   570  1116 ...     0     0     0]] 

Encoded X Test
 [[    8     3   110 ...     0     0     0]
 [   70     1     6 ...     0     0     0]
 [    1    47     3 ...     1  1387   983]
 ...
 [ 1394   197 10723 ...     0     0     0]
 [    1    89  3263 ...    19   626     3]
 [ 1240    73     9 ...     0     0     0]] 

Maximum review length:  130


In [ ]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

40000
10000
40000
10000


In [ ]:
for i in x_train[0:3]:
  print(i)

In [ ]:
for i in x_test[0:3]:
  print(i)

In [ ]:
for i in y_train[0:3]:
  print(i)

1
0
1


In [ ]:
for i in y_test[0:3]:
  print(i)

0
1
0


In [ ]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 32)           2958720   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,983,617
Trainable params: 2,983,617
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint = ModelCheckpoint(
    'models/LSTMtest.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint], validation_data=(x_test,y_test))

Epoch 1/5
313/313 [==============================] - 20s 39ms/step - loss: 0.6908 - accuracy: 0.5226 - val_loss: 0.6353 - val_accuracy: 0.6593

Epoch 00001: accuracy improved from -inf to 0.55773, saving model to models/LSTMtest.h5
Epoch 2/5
313/313 [==============================] - 12s 37ms/step - loss: 0.6246 - accuracy: 0.6684 - val_loss: 0.6538 - val_accuracy: 0.6524

Epoch 00002: accuracy improved from 0.55773 to 0.66562, saving model to models/LSTMtest.h5
Epoch 3/5
313/313 [==============================] - 11s 36ms/step - loss: 0.6238 - accuracy: 0.6660 - val_loss: 0.5239 - val_accuracy: 0.7809

Epoch 00003: accuracy improved from 0.66562 to 0.67905, saving model to models/LSTMtest.h5
Epoch 4/5
313/313 [==============================] - 11s 37ms/step - loss: 0.4405 - accuracy: 0.8213 - val_loss: 0.4351 - val_accuracy: 0.8285

Epoch 00004: accuracy improved from 0.67905 to 0.83220, saving model to models/LSTMtest.h5
Epoch 5/5
313/313 [==============================] - 12s 37ms/s

In [ ]:
y_pred = model.predict_classes(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Correct Prediction: 8706
Wrong Prediction: 1294
Accuracy: 87.06


In [ ]:
loaded_model = load_model('models/LSTMtest.h5')

In [ ]:
review = str(input('Movie Review: '))

Movie Review: good acting and Amazing direction! never bored anytime during movie. engaging i rate it 8/10


In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  good acting and Amazing direction never bored anytime during movie engaging i rate it 
Filtered:  ['good acting amazing direction never bored anytime movie engaging rate ']


In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[   9   43  398  371   41 1032 6259    3 1722  888    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)
if result >= 0.7:
    print('positive')
else:
    print('negative')

[[0.8155729]]
positive
